In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import multiprocessing as mul
from scipy import stats
import pickle
from scipy.integrate import quad
from corner import corner
import pandas as pd
from scipy.stats import gaussian_kde
import dynesty as dyn
from tqdm import tqdm


# data = []
# for i in os.listdir('./outputs/GOF/'):
#     # with open('./outputs/GOF/' + i, 'rb') as f:
#     #     data.append(np.load(f))
#     if i.endswith('_GOF.txt'):
#         data.append([i.replace('_GOF.txt', ''), *np.loadtxt('./outputs/GOF/' + i)])
# df2 = pd.DataFrame(data, columns=['GRB', 'Null', 'Linear', 'Quadratic'])
# df2.index = df2['GRB']
# df2.drop('GRB', axis=1, inplace=True)
# df2.sort_index(inplace=True, ascending=False)
# df2.to_latex('./outputs/GOF/GOF_table.tex', float_format='%.2f', index=True)


In [2]:
GRBs = ['GRB210619B', 'GRB210610B', 'GRB210204A', 'GRB201216C', 'GRB200829A', 'GRB200613A', 'GRB190114C', 'GRB180720B', 'GRB180703A', 'GRB171010A', 'GRB160625B', 'GRB160509A', 'GRB150821A', 'GRB150514A', 'GRB150403A', 'GRB150314A', 'GRB141028A', 'GRB140508A', 'GRB140206A', 'GRB131231A', 'GRB131108A', 'GRB130925A', 'GRB130518A','GRB130427A', 'GRB120119A', 'GRB100728A', 'GRB091003A', 'GRB090926A', 'GRB090618', 'GRB090328', 'GRB081221', 'GRB080916C']
# GRBs = [GRBs[1]]
for grb in GRBs:
    
    grbname = grb + '.txt'
    grbname_wtht_ext = grbname.replace('.txt','')
    grbparam = pd.read_csv('../data/GRBPARAM.csv', index_col=0)

    arr = np.loadtxt('../data/asciidataof_fig1/32lag/'+grbname)
    data = [arr[:,0], arr[:,1], arr[:,2]]
    x = arr[:,0]
    y = arr[:,1]
    yerr = arr[:,2]
    df = pd.read_csv('../data/32lag_err/' + grb + '.txt', sep='\s+', header=None, names=['E_obs', 'E_obs_err', 'lag', 'lag_err'])
    E_err = df['E_obs_err'].values

    #Properties of GRB
    E0 = grbparam[grbname.replace('.txt','')].E0
    E0rest = E0
    Erest = arr[:,0]    #in keV
    z_com = grbparam[grbname.replace('.txt','')].redshift #redshift
    H0=67.36 #Hubble constant km -1 Mpc s -1
    omega_m = 0.315
    omega_l = 1 - omega_m

    lin_conv_fac = 3.0856 * 10**13
    quad_conv_fac = 3.0856 * 10**7
    
    #MODELS

    #NULL model
    def nullhp(E, Eb, alpha1, alpha2, mu, zeta):
        
        eob = (E - E0) / (Eb)
        
        return zeta * (eob**alpha1) * ((0.5 * np.nan_to_num((1 + eob)**(1/mu))) ** ((alpha2 - alpha1) * mu))


    def int_z(z_prime, n):
        integ_fn = lambda z: (1+z)**n / np.sqrt(omega_m * (1+z)**3 + omega_l)
        return quad( integ_fn, a=0, b=z_prime)[0]

    int_z1 = np.asarray(int_z(z_com, 1))
    int_z2 = np.asarray(int_z(z_com, 2))

    #LINEAR model
    def linearhp(E, logEqg, Eb, alpha1, alpha2, mu, zeta):
        
        e0qg = (E - E0) / (10 ** logEqg)
        
        return (lin_conv_fac * e0qg * int_z1)/H0 + nullhp(E, Eb, alpha1, alpha2, mu, zeta)

    #QUADRATIC model
    def quadhp(E, logEqg, Eb, alpha1, alpha2, mu, zeta):
        e0qg = (E**2 - E0 **2) / ((10 ** logEqg)**2)
        
        return 1.5 * (quad_conv_fac * e0qg * int_z2)/H0 + nullhp(E, Eb, alpha1, alpha2, mu, zeta)


    #ERRORS
    
    def ddeltat_dE(E, Eb, alpha1, alpha2, mu, zeta):
        
        eobmu = ((E - E0)/Eb)**(1/mu)
        fac = (alpha1 + ( (alpha2 - alpha1) / (1 + 1/eobmu) ) )/(E - E0)
        # fac = (alpha1 + ( (alpha2 - alpha1) / np.nan_to_num((1 + 1/eobmu) , posinf=np.inf, neginf=-np.inf ) ) )/(E - E0)
    
        return nullhp(E, Eb, alpha1, alpha2, mu, zeta) * fac
    
    def ddeltatdE_LIV_lin(E, logEqg, Eb, alpha1, alpha2, mu, zeta):
        de0qg = 1 / (10 ** logEqg)
        return (lin_conv_fac * de0qg * int_z1)/H0 + ddeltat_dE(E, Eb, alpha1, alpha2, mu, zeta)
    
    def ddeltatdE_LIV_quad(E, logEqg, Eb, alpha1, alpha2, mu, zeta):
        de0qg = 2 * E / ((10 ** logEqg)**2)
        return 1.5 * (quad_conv_fac * de0qg * int_z2)/H0 + ddeltat_dE(E, Eb, alpha1, alpha2, mu, zeta)
    
    
    
    #LOG-LIKELIHOODS
    def loglike_null(theta):
        Eb, alpha1, alpha2, mu, zeta = theta
        
        if alpha1 >= alpha2:
            model = nullhp(x,  Eb, alpha1, alpha2, mu, zeta)
            err = np.sqrt((ddeltat_dE(x, Eb, alpha1, alpha2, mu, zeta) * E_err)**2 + yerr**2)
            
            return sum(stats.norm.logpdf(*args) for args in zip(y,model,err))
        
        return -np.inf

    def loglike_linear(theta):
        logEqg, Eb, alpha1, alpha2, mu, zeta = theta
        
        if alpha1 >= alpha2:
            model = linearhp(x, logEqg, Eb, alpha1, alpha2, mu, zeta)
            err = np.sqrt((ddeltatdE_LIV_lin(x, logEqg, Eb, alpha1, alpha2, mu, zeta) * E_err)**2 + yerr**2)
            
            return sum(stats.norm.logpdf(*args) for args in zip(y,model,err))
        
        return -np.inf

    def loglike_quad(theta):
        logEqg, Eb, alpha1, alpha2, mu, zeta = theta
        
        if alpha1 >= alpha2:
            model = quadhp(x, logEqg, Eb, alpha1, alpha2, mu, zeta)
            err = np.sqrt((ddeltatdE_LIV_quad(x, logEqg, Eb, alpha1, alpha2, mu, zeta) * E_err)**2 + yerr**2)
            
            return sum(stats.norm.logpdf(*args) for args in zip(y,model,err))
        
        return -np.inf    


    #PRIORS
    Ebmax = 5000 #keV
    Ebmin = 0
    alpha1min = -3
    alpha1max = 10
    alpha2min = -10
    alpha2max = 3
    mumin = 0
    mumax = 3
    zetamin = 0
    zetamax = 4

    logeq1min = 0
    logeq1max = 20
    logeq2min = 0
    logeq2max = 15


    #PRIOR DISTRIBUTIONS

    def prior_transform_null(theta):
        Eb, alpha1, alpha2, mu, zeta = theta
        return [Ebmax * Eb + Ebmin, (alpha1max - alpha1min) * alpha1 + alpha1min, (alpha2max - alpha2min) * alpha2 + alpha2min, (mumax - mumin) * mu + mumin, (zetamax - zetamin) * zeta + zetamin]

    def prior_transform_linear(theta):
        logEqg, Eb, alpha1, alpha2, mu, zeta = theta
        return [(logeq1max - logeq1min) * logEqg + logeq1min, Ebmax * Eb + Ebmin, (alpha1max - alpha1min) * alpha1 + alpha1min, (alpha2max - alpha2min) * alpha2 + alpha2min, mumax * mu + mumin, zetamax * zeta + zetamin]

    def prior_transform_quadratic(theta):
        logEqg, Eb, alpha1, alpha2, mu, zeta = theta
        return [(logeq2max - logeq2min) * logEqg + logeq2min, Ebmax * Eb + Ebmin, (alpha1max - alpha1min) * alpha1 + alpha1min, (alpha2max - alpha2min) * alpha2 + alpha2min, mumax * mu + mumin, zetamax * zeta + zetamin]


    #SAMPLING
    nlive = 1024
    sampler0 = 0
    sampler1 = 0
    sampler2 = 0
    results0 = 0
    results1 = 0
    results2 = 0
    
    threesamplers = ['_null_sampler.pkl', '_linear_sampler.pkl', '_quadratic_sampler.pkl']
    sampler0 = dyn.utils.restore_sampler('./outputs/sampler_saves_xerr/' + grb + threesamplers[0])
    sampler1 = dyn.utils.restore_sampler('./outputs/sampler_saves_xerr/' + grb + threesamplers[1])
    sampler2 = dyn.utils.restore_sampler('./outputs/sampler_saves_xerr/' + grb + threesamplers[2])
    results0 = sampler0.results
    results1 = sampler1.results
    results2 = sampler2.results
    nplot = 1000
    E = np.linspace(min(Erest), max(Erest), nplot)
    samples0 = dyn.utils.resample_equal( results0.samples, np.exp(results0.logwt - results0.logz[-1]))
    # samples0 = np.median(samples0, axis=0)
    samples0 = samples0[np.argmax(results0.logl)]

    samples1 = dyn.utils.resample_equal( results1.samples, np.exp(results1.logwt - results1.logz[-1]))
    # samples1 = np.median(samples1, axis=0)
    samples1 = samples1[np.argmax(results1.logl)]

    samples2 = dyn.utils.resample_equal( results2.samples, np.exp(results2.logwt - results2.logz[-1]))
    # samples2 = np.median(samples2, axis=0)
    samples2 = samples2[np.argmax(results2.logl)]
    null_fit = [nullhp(E[i], samples0[0], samples0[1], samples0[2], samples0[3], samples0[4]) for i in range(nplot)]
    liv_lin_fit = [linearhp(E[i], samples1[0], samples1[1], samples1[2], samples1[3], samples1[4], samples1[5]) for i in range(nplot)]
    liv_quad_fit = [quadhp(E[i], samples2[0], samples2[1], samples2[2], samples2[3], samples2[4], samples2[5]) for i in range(nplot)]

    plt.figure()
    plt.errorbar(Erest, y, xerr=E_err, yerr=yerr, fmt='o', color='black', label='data')
    plt.plot(E, null_fit, label='Null fit')
    plt.plot(E, liv_lin_fit,label='Linear fit')
    plt.plot(E, liv_quad_fit, label='Quadratic fit')
    plt.xscale('log')
    # plt.yscale('log')
    plt.ylim(min(y) - max(abs(yerr)), max(y) + max(abs(yerr)))
    # plt.ylim(-200, 20)
    plt.legend()
    plt.xlabel('E (keV)')
    plt.ylabel('lag (s)')
    plt.title(grbname_wtht_ext)
    plt.savefig('./outputs/fits_xerr/' + grbname_wtht_ext + '_fit_logE_xerr.png', facecolor='white')
    plt.close()

    def chi2_gof(x, y, yerr, n, *fit_func_args):

        
        if n == 0:
            err = np.sqrt((ddeltat_dE(x, *fit_func_args) * E_err)**2 + yerr**2)
            fit_func = nullhp
            return np.sum(((y - fit_func(x, *fit_func_args))/err)**2)/(len(y) - len(fit_func_args))
        
        elif n == 1:
            err = np.sqrt((ddeltatdE_LIV_lin(x, *fit_func_args) * E_err)**2 + yerr**2)
            fit_func = linearhp
            return np.sum(((y - fit_func(x, *fit_func_args))/err)**2)/(len(y) - len(fit_func_args))
        
        elif n == 2:
            err = np.sqrt((ddeltatdE_LIV_quad(x, *fit_func_args) * E_err)**2 + yerr**2)
            fit_func = quadhp
            return np.sum(((y - fit_func(x, *fit_func_args))/err)**2)/(len(y) - len(fit_func_args))


    gof_null = chi2_gof(Erest, y, yerr, 0, samples0[0], samples0[1], samples0[2], samples0[3], samples0[4])
    gof_lin = chi2_gof(Erest, y, yerr, 1, samples1[0], samples1[1], samples1[2], samples1[3], samples1[4], samples1[5])
    gof_quad = chi2_gof(Erest, y, yerr, 2, samples2[0], samples2[1], samples2[2], samples2[3], samples2[4], samples2[5])

    print(grb, gof_null, gof_lin, gof_quad)
    with open('./outputs/GOF_xerr/' + grb + '_GOF.txt', 'w') as f:
        f.write(str(gof_null) + '\n')
        f.write(str(gof_lin) + '\n')
        f.write(str(gof_quad) + '\n')
        
    f.close()
    f = []
    
    
    


/tmp/ipykernel_337599/736722860.py:36: RuntimeWarning: overflow encountered in scalar power
  return zeta * (eob**alpha1) * ((0.5 * np.nan_to_num((1 + eob)**(1/mu))) ** ((alpha2 - alpha1) * mu))
/tmp/ipykernel_337599/736722860.py:65: RuntimeWarning: divide by zero encountered in divide
  fac = (alpha1 + ( (alpha2 - alpha1) / (1 + 1/eobmu) ) )/(E - E0)
/tmp/ipykernel_337599/736722860.py:36: RuntimeWarning: overflow encountered in power
  return zeta * (eob**alpha1) * ((0.5 * np.nan_to_num((1 + eob)**(1/mu))) ** ((alpha2 - alpha1) * mu))


GRB210619B 3.6018692598892486 3.708849354537578 2.5483726712635555
GRB210610B 0.9992453353289776 0.8713467576861592 1.1827236548066624
GRB210204A 5.896571141174779 6.086365015451856 6.450046569587255
GRB201216C 1.46473955519522 1.0578018418157933 1.0592992113130226


/tmp/ipykernel_337599/736722860.py:36: RuntimeWarning: overflow encountered in scalar power
  return zeta * (eob**alpha1) * ((0.5 * np.nan_to_num((1 + eob)**(1/mu))) ** ((alpha2 - alpha1) * mu))
/tmp/ipykernel_337599/736722860.py:65: RuntimeWarning: divide by zero encountered in divide
  fac = (alpha1 + ( (alpha2 - alpha1) / (1 + 1/eobmu) ) )/(E - E0)
/tmp/ipykernel_337599/736722860.py:36: RuntimeWarning: overflow encountered in power
  return zeta * (eob**alpha1) * ((0.5 * np.nan_to_num((1 + eob)**(1/mu))) ** ((alpha2 - alpha1) * mu))


GRB200829A 6.280631342488358 6.7412277095948925 6.85818676889503
GRB200613A 0.5955394492496814 0.6958664041516875 1.1797831956741938
GRB190114C 3.520091315050045 3.6296567390531598 3.7967031763198484
GRB180720B 0.950129572345794 0.9050361418382835 1.2238314952620344
GRB180703A 9.010889468118194 9.928550357749845 10.801660974597098
GRB171010A 0.5652662790098175 0.7218433419950845 0.6280311352016945
GRB160625B 8.00035305070902 8.90683010765008 4.369096896442039
GRB160509A 0.7145261859754096 0.8992873370695605 0.8594390135843123
GRB150821A 0.9531085554862142 1.230650415546824 0.8963539842484441
GRB150514A 1.110553444185368 1.1857596430029549 1.293293824121214
GRB150403A 0.8313035646230595 0.9538688585272783 1.1416302587314597
GRB150314A 3.1973007593522893 3.85792481512789 2.926172233568963
GRB141028A 0.7780668034527485 2.0353897504344034 0.8877011459269802
GRB140508A 0.8265232807751062 0.9671716297593457 1.3263762743994165
GRB140206A 1.4189966984483167 1.6211615437093005 1.970382083878642

In [3]:
1/np.inf

0.0

In [5]:
1/np.nan_to_num(np.inf)

5.562684646268003e-309